In [14]:
#| default_exp sandbox

In [15]:
#| hide
from nbdev.showdoc import *

In [16]:
#| hide
import nbdev; nbdev.nbdev_export()

In [17]:
#| hide
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [31]:
from dotenv import load_dotenv
import os
load_dotenv("../.env")

True

In [ ]:
os.environ['HTTP_PROXY'] = "http://127.0.0.1:20171"
os.environ['HTTPS_PROXY'] = "http://127.0.0.1:20171"
os.environ['http_proxy'] = "http://127.0.0.1:20171"
os.environ['https_proxy'] = "http://127.0.0.1:20171"

In [32]:
os.getenv('GC_QUOTE_API_CREDENTIALS')
os.getenv('HTTP_PROXY')
os.getenv('HTTPS_PROXY')

'res/quote-api-365206-39d39b2fde07.json'

'http://127.0.0.1:20171'

'http://127.0.0.1:20171'

In [34]:
from dotenv import dotenv_values
config = dotenv_values("../.env")
key = config.get('GC_QUOTE_API_CREDENTIALS')

In [35]:
from google.cloud import bigquery
import pandas_gbq
from google.oauth2 import service_account
credentials = service_account.Credentials.from_service_account_file("../" + key)

In [36]:
client = bigquery.Client(credentials=credentials)

QUERY = """
		select *
		from `quote-api-365206.report.ride_dispatch_v` r
		where dispatch_amount_net_usd is not null
        limit 200
		"""

In [37]:
QUERY_DISINCT = """
		select count (*)
		from `quote-api-365206.report.ride_dispatch_v` r
		where trip_type is null
        limit 200
		"""

In [38]:
!echo $HTTP_PROXY

http://127.0.0.1:20171


In [26]:
#query_job = client.query(QUERY)

In [39]:

df = pandas_gbq.read_gbq(QUERY,credentials=credentials)

KeyboardInterrupt: 

In [12]:

df_trip_type = pandas_gbq.read_gbq(QUERY_DISINCT,credentials=credentials)

Downloading: 100%|██████████|


In [26]:

df.head()

,ride_id,trip_no,dispatch_id,trip_type,ride_stat,dispatch_stat,partner_tran_id,partner_id,from_fleet_id_as_customer,pickup_date,...,has_complaint,has_fleet_complaint,has_penalty_complaint,has_ops_complaint,gps_ad_check,gps_cns_check,score,incoming_dispatch_info_last_updated_at,dispatch_info_last_updated_at,etl_time
0,2324292,1,345056,None,Accepted,At destination,161955,1,16,2022-08-01,...,0,0,0,0,0,0,None,2022-07-08 20:18:48+00:00,2022-08-01 23:58:01+00:00,2024-08-29 23:05:33.393026+00:00
1,2398953,0,518874,None,Accepted,Customer no show,237411,1,16,2022-11-24,...,1,1,0,0,0,0,None,2022-11-24 09:28:34+00:00,2022-11-24 09:03:50+00:00,2024-08-29 23:05:33.393026+00:00
2,2378500,0,472749,None,Accepted,At destination,216589,1,16,2022-10-21,...,1,0,1,1,0,0,None,2023-01-10 16:48:45+00:00,2023-01-10 01:08:03+00:00,2024-08-29 23:05:33.393026+00:00
3,2382836,0,482760,None,Accepted,Customer no show,221063,1,16,2022-11-25,...,1,1,0,0,0,0,None,2022-12-02 10:25:19+00:00,2022-11-25 12:12:22+00:00,2024-08-29 23:05:33.393026+00:00
4,3316247,0,2340475,point2point,Pending,At destination,1162413,1,16,2024-03-28,...,0,0,0,0,0,0,None,2024-04-18 05:23:39+00:00,2024-04-18 05:24:06+00:00,2024-08-29 23:05:33.393026+00:00


In [14]:
QUERY_ROUTE = """
        SELECT r.*, p1.lat from_lat, p1.lng from_lng, p2.lat to_lat, p2.lng to_lng
        FROM `report.ride_dispatch_v` r
        left JOIN `report.dim_place` p1
        ON r.pickup_place_id=p1.id
        INNER JOIN `report.dim_place` p2
        ON r.pickup_place_id=p2.id
        ORDER BY ride_id DESC
        limit 2;
		"""


In [15]:
df_route = pandas_gbq.read_gbq(QUERY_ROUTE,credentials=credentials)

Downloading: 100%|██████████|


In [16]:
df_route.head()

,ride_id,trip_no,dispatch_id,trip_type,ride_stat,dispatch_stat,partner_tran_id,partner_id,from_fleet_id_as_customer,pickup_date,...,gps_ad_check,gps_cns_check,score,incoming_dispatch_info_last_updated_at,dispatch_info_last_updated_at,etl_time,from_lat,from_lng,to_lat,to_lng
0,3766119,0,3218599,point2point,Pending,None,1584248,2,18,2024-09-03,...,0,0,None,2024-08-29 22:51:31+00:00,NaT,2024-08-29 23:05:33.393026+00:00,9.547790000,100.062000000,9.547790000,100.062000000
1,3766118,0,3218598,point2point,Pending,None,1584247,2,18,2024-09-06,...,0,0,None,2024-08-29 22:51:12+00:00,NaT,2024-08-29 23:05:33.393026+00:00,25.796914000,-80.288455000,25.796914000,-80.288455000


In [17]:
df_route.dtypes

ride_id                                                 Int64
trip_no                                                 Int64
dispatch_id                                             Int64
trip_type                                              object
ride_stat                                              object
dispatch_stat                                          object
partner_tran_id                                         Int64
partner_id                                              Int64
from_fleet_id_as_customer                               Int64
pickup_date                                            dbdate
pickup_time                                            dbtime
pickup_datetime                           datetime64[us, UTC]
pickup_local_datetime                          datetime64[us]
pickup_local_datetime_str                              object
pickup_place_id                                         Int64
dropoff_place_id                                        Int64
passenge

In [2]:
import pandas as pd
import pandera as pa

# data to validate
df = pd.DataFrame({
    "column1": [1, 4, 0, 10, 9, 11],
    "column2": [-1.3, -1.4, -2.9, -10.1, -20.4, -2.0],
    "column3": ["value_1", "value_2", "value_3", "value_2", "value_1", "value_2"],
})

# define schema
schema = pa.DataFrameSchema({
    "column1": pa.Column(int, checks=pa.Check.le(10)),
    "column2": pa.Column(float, checks=pa.Check.lt(-1.2)),
    "column3": pa.Column(str, checks=[
        pa.Check.str_startswith("value_"),
        # define custom checks as functions that take a series as input and
        # outputs a boolean or boolean Series
        pa.Check(lambda s: s.str.split("_", expand=True).shape[1] == 2)
    ]),
})

validated_df = schema(df)
print(validated_df)

SchemaError: Column 'column1' failed element-wise validator number 0: less_than_or_equal_to(10) failure cases: 11

In [3]:
from pandera.typing import Series

class Schema(pa.DataFrameModel):

    column1: int = pa.Field(le=10)
    column2: float = pa.Field(lt=-1.2)
    column3: str = pa.Field(str_startswith="value_")

    @pa.check("column3")
    def column_3_check(cls, series: Series[str]) -> Series[bool]:
        """Check that column3 values have two elements after being split with '_'"""
        return series.str.split("_", expand=True).shape[1] == 2

Schema.validate(df)

SchemaError: Column 'column1' failed element-wise validator number 0: less_than_or_equal_to(10) failure cases: 11

In [155]:
from datetime import datetime

def validate_datetime(date_string, format_string):
    """
    Validates if a string is a valid datetime according to the given format.
    """

    try:
        datetime.strptime(date_string, format_string)
        return True
    except ValueError:
        return False
    
def validate_datetime_in_iso_format(date_text):
        try:
            datetime.fromisoformat(date_text)
            return True
        except ValueError:
            # raise ValueError("Incorrect data format, should be YYYY-MM-DD")
            return False
# Example usage
date_strings = [
    "2023-12-25 24:00:00",
    "2023-12-25 12:10:00",
    "2023-12-25 23:00:00",
    "2011-11-04",
    "20111104",
    "2011-11-04T00:05:23",
    "2011-11-04T00:05:23Z",
    "2011-11-04T00:05:23.283185",
    "2011-11-04T00:05:23.283185+08:00",
    "20111104T000523",
    "20111104T000523.283185",
    "2011-11-04T00:05:23+08:00",
    ]
format_string = "%Y-%m-%d"

for s in date_strings:
    if validate_datetime_in_iso_format(s):
        print(f"{s} is Valid datetime string")
    else:
        print(f"{s} is Invalid datetime string")

2023-12-25 24:00:00 is Invalid datetime string
2023-12-25 12:10:00 is Valid datetime string
2023-12-25 23:00:00 is Valid datetime string
2011-11-04 is Valid datetime string
20111104 is Valid datetime string
2011-11-04T00:05:23 is Valid datetime string
2011-11-04T00:05:23Z is Valid datetime string
2011-11-04T00:05:23.283185 is Valid datetime string
2011-11-04T00:05:23.283185+08:00 is Valid datetime string
20111104T000523 is Valid datetime string
20111104T000523.283185 is Valid datetime string
2011-11-04T00:05:23+08:00 is Valid datetime string


In [172]:
df_datetime_str = pd.DataFrame(date_strings)
df_datetime_str.columns = ['datetime']
# df_datetime_str
df_datetime_str['validity'] = df_datetime_str.apply(lambda x: validate_datetime_in_iso_format(x.iloc[0]), axis=1)
df_datetime_str


,datetime,validity
0,2023-12-25 24:00:00,False
1,2023-12-25 12:10:00,True
2,2023-12-25 23:00:00,True
3,2011-11-04,True
4,20111104,True
5,2011-11-04T00:05:23,True
6,2011-11-04T00:05:23Z,True
7,2011-11-04T00:05:23.283185,True
8,2011-11-04T00:05:23.283185+08:00,True
9,20111104T000523,True


In [38]:
df_datetime = df_datetime_str[df_datetime_str.apply(lambda x: validate_datetime_in_iso_format(x.iloc[0]), axis=1)]
df_datetime


,datetime,validity
1,2023-12-25 12:10:00,True
2,2023-12-25 23:00:00,True
3,2011-11-04,True
4,20111104,True
5,2011-11-04T00:05:23,True
6,2011-11-04T00:05:23Z,True
7,2011-11-04T00:05:23.283185,True
8,20111104T000523,True
9,20111104T000523.283185,True
10,2011-11-04T00:05:23+08:00,True


In [44]:
df_datetime['datetime']

1            2023-12-25 12:10:00
2            2023-12-25 23:00:00
3                     2011-11-04
4                       20111104
5            2011-11-04T00:05:23
6           2011-11-04T00:05:23Z
7     2011-11-04T00:05:23.283185
8                20111104T000523
9         20111104T000523.283185
10     2011-11-04T00:05:23+08:00
Name: datetime, dtype: object

In [55]:
# df_datetime_str.apply(lambda x: validate_datetime_in_iso_format(x.iloc[0]), axis=1)
df_datetime1 = df_datetime.apply(lambda x: datetime.fromisoformat(x['datetime']), axis=1)
df_datetime1.name = 'datetime'
df_datetime1


1            2023-12-25 12:10:00
2            2023-12-25 23:00:00
3            2011-11-04 00:00:00
4            2011-11-04 00:00:00
5            2011-11-04 00:05:23
6      2011-11-04 00:05:23+00:00
7     2011-11-04 00:05:23.283185
8            2011-11-04 00:05:23
9     2011-11-04 00:05:23.283185
10     2011-11-04 00:05:23+08:00
Name: datetime, dtype: object

In [71]:
pd.to_datetime(df_datetime1[7:9])

8   2011-11-04 00:05:23.000000
9   2011-11-04 00:05:23.283185
Name: datetime, dtype: datetime64[ns]

In [87]:
import pytz
def get_timezone_abbreviation(timezone_name):
    timezone = pytz.timezone(timezone_name)
    now = datetime.now(timezone)
    return now.strftime("%Z")

def validate_timezone_in_iana(timezone_name):
     return timezone_name in pytz.all_timezones

time_zone_strings = [
    'Asia/Shanghai',
    'Asia/Mumbai',
    'America/New_York',
    'Europe/London', 
    'Eastern Standard Time', 
    'US/Eastern', 
    'America/Los_Angeles', 
    'Asia/Kolkata', 
    'Europe/London'
]

In [180]:
df_timezone_str = pd.DataFrame(time_zone_strings)
df_timezone_str.columns = ['timezone']
# df_datetime_str
df_timezone_str['validity'] = df_timezone_str.apply(lambda x: validate_timezone_in_iana(x.iloc[0]), axis=1)
df_timezone_str


,timezone,validity
0,Asia/Shanghai,True
1,Asia/Mumbai,False
2,America/New_York,True
3,Europe/London,True
4,Eastern Standard Time,False
5,US/Eastern,True
6,America/Los_Angeles,True
7,Asia/Kolkata,True
8,Europe/London,True


In [181]:

df_timezone = df_timezone_str[df_timezone_str.apply(lambda x: validate_timezone_in_iana(x.iloc[0]), axis=1)]
df_timezone


,timezone,validity
0,Asia/Shanghai,True
2,America/New_York,True
3,Europe/London,True
5,US/Eastern,True
6,America/Los_Angeles,True
7,Asia/Kolkata,True
8,Europe/London,True


In [188]:
df_abbr = df_timezone.apply(lambda x: get_timezone_abbreviation(x.loc['timezone']), axis=1)
df_abbr.name = 'abbr'
df_abbr

0    CST
2    EDT
3    BST
5    EDT
6    PDT
7    IST
8    BST
Name: abbr, dtype: object

In [183]:
df_timezone

,timezone,validity
0,Asia/Shanghai,True
2,America/New_York,True
3,Europe/London,True
5,US/Eastern,True
6,America/Los_Angeles,True
7,Asia/Kolkata,True
8,Europe/London,True


In [186]:
# df_timezone_str['validity'] = df_timezone_str.apply(lambda x: validate_timezone_in_iana(x.iloc[0]), axis=1)
# df_timezone['abbr'] = df_timezone.apply(lambda x: get_timezone_abbreviation(x.iloc[0]), axis=1)
# df_timezone.apply(lambda x: get_timezone_abbreviation(x.iloc[0]), axis=1)
df_timezone.loc[:,'abbr'] = df_abbr
df_timezone
# df_timezone


,timezone,validity,abbr
0,Asia/Shanghai,True,CST
2,America/New_York,True,EDT
3,Europe/London,True,BST
5,US/Eastern,True,EDT
6,America/Los_Angeles,True,PDT
7,Asia/Kolkata,True,IST
8,Europe/London,True,BST


In [200]:

df_utc_offset = (df_timezone.apply(lambda x: datetime.now(pytz.timezone(x.loc['timezone'])), axis=1)
                            .apply(lambda x: x.strftime('%z')))
df_utc_offset.name = 'utc_offset'
df_utc_offset


0    +0800
2    -0400
3    +0100
5    -0400
6    -0700
7    +0530
8    +0100
Name: utc_offset, dtype: object

In [201]:

df_timezone.loc[:,'utc_offset'] = df_utc_offset
df_timezone


,timezone,validity,abbr,utc_offset
0,Asia/Shanghai,True,CST,+0800
2,America/New_York,True,EDT,-0400
3,Europe/London,True,BST,+0100
5,US/Eastern,True,EDT,-0400
6,America/Los_Angeles,True,PDT,-0700
7,Asia/Kolkata,True,IST,+0530
8,Europe/London,True,BST,+0100


In [171]:
for tz in df_timezone.loc[:,'timezone']:
    print(f"{tz}: {datetime.now(pytz.timezone(tz)).strftime('%Z')}, {datetime.now(pytz.timezone(tz)).strftime('%z')}")

Asia/Shanghai: CST, +0800
America/New_York: EDT, -0400
Europe/London: BST, +0100
US/Eastern: EDT, -0400
America/Los_Angeles: PDT, -0700
Asia/Kolkata: IST, +0530
Europe/London: BST, +0100
